In [30]:
from imaplib import Time2Internaldate
import torch
import time

#用于构建NN
import torch.nn as nn

#需要用到这个库里面的激活函数
import torch.nn.functional as F

#用于构建优化器
import torch.optim as optim

#用于初始化数据
from torchvision import datasets, transforms

#用于分布式训练
import syft as sy
 
hook = sy.TorchHook(torch)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [31]:
Bob = sy.VirtualWorker(hook,id='Bob')
Alice = sy.VirtualWorker(hook,id='Alice')


In [32]:
class Arguments():
    def __init__(self):
        self.batch_size = 1
        self.test_batch_size = 100
        self.epochs = 1
        self.lr = 0.01
        self.momentum = 0.5
        self.seed = 1
        self.log_interval = 1
        self.save_model = True
#实例化参数类
args = Arguments()

#固定化随机数种子，使得每次训练的随机数都是固定的
torch.manual_seed(args.seed)


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data as Data
import matplotlib.pyplot as plt
import seaborn as sns
import os
process_dataset_x= pd.read_csv('/home/ysy/ysy/Fed-ReKD-dirs/process_dataset_x.csv',index_col=0)
process_dataset_y= pd.read_csv('/home/ysy/ysy/Fed-ReKD-dirs/process_dataset_y.csv',index_col=0)
XX= process_dataset_x
YY= process_dataset_y['logerror']
target=YY
scale=StandardScaler()
x=scale.fit_transform(XX) 
del XX,YY
x=np.array(x)
target=np.array(target)
import torch
import numpy as np
 
from tqdm import tqdm
# target
train_dataset = []
train_loader = []
from torch.utils.data import TensorDataset, DataLoader
train_dataset=TensorDataset(torch.tensor(x,dtype=torch.float),
                                    torch.tensor(target,dtype=torch.float))
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True)
#定义联邦训练数据集，定义转换器为 x=(x-mean)/标准差
fed_dataset_Bob = train_dataset

#定义数据加载器，shuffle是采用随机的方式抽取数据
fed_loader_Bob = torch.utils.data.DataLoader(fed_dataset_Bob,batch_size=args.batch_size,shuffle=True)

#定义联邦训练数据集，定义转换器为 x=(x-mean)/标准差
fed_dataset_Alice = train_dataset

#定义数据加载器，shuffle是采用随机的方式抽取数据
fed_loader_Alice = torch.utils.data.DataLoader(fed_dataset_Alice,batch_size=args.batch_size,shuffle=True)

#定义测试集
test_dataset = train_dataset

#定义测试集加载器
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=args.test_batch_size,shuffle=True)


In [40]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Sequential(nn.Linear(in_features=81, out_features=810, bias=True),
                                  nn.Dropout(0.7),
                                  nn.Tanh()
                                  )
        
        self.hidden2 = nn.Sequential(nn.Linear(in_features=810, out_features=162, bias=True),
                                  nn.Dropout(0.7),
                                  nn.Tanh()
                                  )
        
        self.hidden3 = nn.Sequential(nn.Linear(in_features=162, out_features=54, bias=True),
                                  nn.Dropout(0.6),
                                  nn.Tanh()
                                  )
        
        self.predict = nn.Sequential(nn.Linear(in_features=54, out_features=1, bias=True),
                                        nn.Tanh())
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = self.predict(x)
        return output
model=MLP().to(device)
print(model)

MLP(
  (hidden1): Sequential(
    (0): Linear(in_features=81, out_features=810, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): Tanh()
  )
  (hidden2): Sequential(
    (0): Linear(in_features=810, out_features=162, bias=True)
    (1): Dropout(p=0.7, inplace=False)
    (2): Tanh()
  )
  (hidden3): Sequential(
    (0): Linear(in_features=162, out_features=54, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Tanh()
  )
  (predict): Sequential(
    (0): Linear(in_features=54, out_features=1, bias=True)
    (1): Tanh()
  )
)


In [48]:
def fedavg_updata_weight(model:MLP, Alice_model:MLP, Bob_model:MLP, num:int):
    """
    训练中需要修改的参数如下，对以下参数进行avg
    # conv1.weight
    # conv1.bias
    # conv2.weight
    # conv2.bias
    fc1.weight
    fc1.bias
    fc2.weight
    fc2.bias
    """
    # model.fc1.weight.set_((Bob_model.fc1.weight.data+Alice_model.fc1.weight.data)/num)
    # model.fc1.bias.set_((Bob_model.fc1.bias.data+Alice_model.fc1.bias.data)/num)
    # model.fc2.weight.set_((Bob_model.fc2.weight.data+Alice_model.fc2.weight.data)/num)
    # model.fc2.bias.set_((Bob_model.fc2.bias.data+Alice_model.fc2.bias.data)/num)

    # model.fc3.weight.set_((Bob_model.fc3.weight.data+Alice_model.fc3.weight.data)/num)
    # model.fc3.bias.set_((Bob_model.fc3.bias.data+Alice_model.fc3.bias.data)/num)
    # model.fc4.weight.set_((Bob_model.fc4.weight.data+Alice_model.fc4.weight.data)/num)
    # model.fc4.bias.set_((Bob_model.fc4.bias.data+Alice_model.fc4.bias.data)/num)
    


In [45]:
def train(model:MLP,fed_loader:torch.utils.data.DataLoader):
    Bob_model = MLP()
    Alice_model = MLP()
    #定义Bob的优化器
    Bob_opt = optim.SGD(Bob_model.parameters(), lr = args.lr)
    #定义Alice的优化器
    Alice_opt = optim.SGD(Alice_model.parameters(), lr = args.lr)

    model.train()
    Bob_model.train()
    Alice_model.train()

    Bob_model.send(Bob)
    Alice_model.send(Alice)
    for epoch in range(1, args.epochs +1):
        #传递模型
        Alice_loss = 0
        Bob_loss = 0
        #模拟Bob训练数据
        for epoch_ind, (data, target) in enumerate(fed_loader):
            data = data.send(Bob)
            target = target.send(Bob)

            Bob_opt.zero_grad()
            pred = Bob_model(data)
            Bob_loss = F.nll_loss(pred,target)
            Bob_loss.backward()
            Bob_opt.step()

            if(epoch_ind%50==0):
                print("There is epoch:{} epoch_ind:{} in Bob loss:{:.6f}".format(epoch,epoch_ind,Bob_loss.get().data.item()))

        #模拟Alice训练模型
        for epoch_ind, (data, target) in enumerate(fed_loader):
            data = data.send(Alice)
            target = target.send(Alice)

            Alice_opt.zero_grad()
            pred = Alice_model(data)
            Alice_loss = F.nll_loss(pred,target)
            Alice_loss.backward()
            Alice_opt.step()
            if(epoch_ind%50==0):
                print("There is epoch:{} epoch_ind:{} in Alice loss:{:.6f}".format(epoch,epoch_ind,Alice_loss.get().data.item()))


        with torch.no_grad():
            Bob_model.get()
            Alice_model.get()
            #更新权重
            fedavg_updata_weight(model,Alice_model,Bob_model,2)

        if epoch % args.log_interval==0:
            #获得loss
            #模型的loss
            # pred = model(fed_loader)
            # Loss = F.nll_loss(pred,target)
            print("Bob in train:")
            test(Bob_model,test_loader)
            print("Alice in train:")
            test(Alice_model,test_loader)
            print("model in train:")
            test(model,test_loader)


In [46]:
#定义测试函数
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output=model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(1, keepdim=True)
            correct+= pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)    
        print('\nTest set : Average loss : {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100.* correct / len(test_loader.dataset)))


In [49]:

if __name__ == '__main__':
    model = MLP()
    start=time.clock()
    train(model, fed_loader_Bob)
    mid=time.clock()
    print("model in test:")
    test(model, test_loader)
    end=time.clock()

    time1 = mid - start
    time2 = end - mid
    print("训练时间：{}h{}m{}s 测试时间为：{}h{}m{}s".format(time1//60//60,time1//60,time1%60,time2//60//60,time2//60,time2%60))

    if(args.save_model):
        torch.save(model.state_dict(),"MLP.weight")


RuntimeError: expected scalar type Long but found Float